# 处理腾讯新闻数据，为article_comment模型提供数据

1、加载train \ test \ dev 数据 <br>
2、对train \ test \ dev 过滤掉脏数据<br>
3、对train数据集进行正负平衡<br>
4、对 train\dev\test 数据集的article、title进行分词, 得到文字版的 train\dev\test 数据集<br>
5.1 根据train数据集构建词典（可选）<br>
5.2 根据pretrain好的词向量构建词典（可选）<br>
6、将 train, dev, test 数据转换成模型需要的id格式<br>

In [1]:
import json
import collections
import pickle

import pandas as pd
from tqdm.autonotebook import tqdm
import jieba
import os

D:\software\Anaconda3\envs\tf\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## 1 加载 train \ test \ dev 数据 
1、发现数据中存在脏数据。 
* 比如article==“每日更新超多有趣漫画，喜欢请关注”，“更多趣味内涵内容，敬请关注微信公众号：漫画精选集（jx2018mh）”，“文章已被原作者删除，目前无法查看。”

In [2]:
def load_data(filename):
    data = []
    with open(filename, encoding='utf-8') as fin:
        for line in fin:
            item = json.loads(line)
            title = item['title_es'].strip()
            article = item['body'].strip()
            url = item['url']
            uid = url.split('/')[-1]
            for comment in item['comment']:
                comment_content = ''.join(comment[0])
                upvote = int(comment[1])
                data.append({
                    'article_id': uid,
                    'title': title,
                    'article': article,
                    'comment': comment_content,
                    'upvote': upvote
                })
    return pd.DataFrame(data)

filename_train = r'D:\data\comment_generation\article_commenting_learning_comment_generation\newdata.train.json'
filename_dev = r'D:\data\comment_generation\article_commenting_learning_comment_generation\newdata.dev.json'
filename_test = r'D:\data\comment_generation\article_commenting_learning_comment_generation\newdata.test.json'

In [3]:
df_train = load_data(filename_train)
df_dev = load_data(filename_dev)
df_test = load_data(filename_test)

In [4]:
print("清洗前训练集size：",len(df_train))
print("清洗前验证集size：",len(df_dev))
print("清洗前测试集size：",len(df_test))

清洗前训练集size： 5131167
清洗前验证集size： 133739
清洗前测试集size： 43833


In [5]:
df_train["article"].describe()

count               5131167
unique               167851
top       文章已被原作者删除，目前无法查看。
freq                 604000
Name: article, dtype: object

In [6]:
df_train[df_train["article"] == ""].head()

,article,article_id,comment,title,upvote
3313,,20170419A02D6Z00,那么 多 解决办法 ， 他 却 选择 了 最 愚蠢 的 一个 。,,0
3314,,20170419A02D6Z00,超 不过 24 小时,,0
3315,,20170419A02D6Z00,那个 儿货发 的 ， 也 没个 照片 ！,,0
3316,,20170419A02D6Z00,这是 旧闻 。 案子 都 破 了 还 悬赏 缉凶 ？,,0
3317,,20170419A02D6Z00,缉拿 谁 呀 ， 你 发个 照片,,0


##  2 过滤掉 train \ test \ dev 中的脏数据

In [7]:
def filter_data(df):
    mask = ((df['article'] == "") | (df['article'] == 'null\n') | (df['article'] == 'null') | (df['title'] == 'null') | (df['title'].isnull()) | (df["article"] == "更多趣味内涵内容，敬请关注微信公众号：漫画精选集（jx2018mh）" )| (df["article"] == "每日更新超多有趣漫画，喜欢请关注") |(df["article"] == "文章已被原作者删除，目前无法查看。" ))
    df = df[~mask]
    return df

cleaned_df_train = filter_data(df_train)
cleaned_df_dev = filter_data(df_dev)
cleaned_df_test = filter_data(df_test)
print("清洗后训练集size：",len(cleaned_df_train))
print("清洗后验证集size：",len(cleaned_df_dev))
print("清洗后测试集size：",len(cleaned_df_test))

清洗后训练集size： 4511253
清洗后验证集size： 118665
清洗后测试集size： 37912


In [8]:
print(cleaned_df_train["comment"][0])

就服 你 了 满屏


In [9]:
cleaned_df_train[cleaned_df_train["article"]==""]

,article,article_id,comment,title,upvote


In [12]:
cleaned_df_train["article"].describe()

count                                               4511253
unique                                               167846
top       星座简介：白羊座（Aries）出生日期：阳历3月21日-4月19日/4月20日，黄道十二星座...
freq                                                    345
Name: article, dtype: object

## 3 对train数据集进行正负平衡

In [13]:
grouped = cleaned_df_train.groupby(['article_id'])

def sample_neg(grouped_df, count):
    neg_samples = grouped_df[grouped_df['upvote'] < 10]
    neg_sample_len_lte_100 = neg_samples[neg_samples['comment'].str.len() <= 100]
    neg_sample_len_gt_100 = neg_samples[neg_samples['comment'].str.len() > 100]
    
    if len(neg_sample_len_lte_100) >= count:
        return neg_sample_len_lte_100.sample(n=count, random_state=1)
    else:
        lte_100_samples = neg_sample_len_lte_100
        gt_100_count = min(len(neg_sample_len_gt_100), count - len(neg_sample_len_lte_100))
        gt_100_samples = neg_sample_len_gt_100.sample(n=gt_100_count, random_state=1)
        return pd.concat([lte_100_samples, gt_100_samples])

samples = []
for name, grouped_df in tqdm(grouped):
    grouped_pos_df = grouped_df[grouped_df['upvote'] >= 10]
    pos_count = len(grouped_pos_df)
    if pos_count == 0:
        continue
    grouped_neg_df = sample_neg(grouped_df, pos_count)
    samples.append(grouped_pos_df)
    samples.append(grouped_neg_df)

sampled_df_train = pd.concat(samples)

将正负均衡后的训练集数据保存到本地，以后直接从本地读取即可

In [14]:
def save(df, save_to):
    df.to_csv(save_to, sep='\t', encoding='utf-8')

def load(filename):
    return pd.read_csv(filename, sep='\t', encoding='utf-8')

if not os.path.exists('./train_balance.csv'):
    save(sampled_df_train, './train_balance.csv')
# sampled_df_train = load('./train_balance.csv')
print("正负均衡后训练集的size：",len(sampled_df_train))

正负均衡后训练集的size： 707655


## 4 对 train\dev\test 数据集的article、title进行分词, 得到文字版的 train\dev\test 数据集

In [15]:
class Tokenizer:
    def __init__(self):
        self._cache = dict()
        
    def cut(self, text):
        if text in self._cache:
            return self._cache[text]
        wb = list(jieba.cut(text))
        wb = ' '.join(wb)
        self._cache[text] = wb
        return wb

tokenizer = Tokenizer()
tqdm.pandas(desc='word break:')

In [16]:
# 对trian分词
sampled_df_train['title_wb'] = sampled_df_train['title'].progress_apply(tokenizer.cut)
sampled_df_train['article_wb'] = sampled_df_train['article'].progress_apply(tokenizer.cut)
# 对dev分词
cleaned_df_dev['title_wb'] = cleaned_df_dev['title'].progress_apply(tokenizer.cut)
cleaned_df_dev['article_wb'] = cleaned_df_dev['article'].progress_apply(tokenizer.cut)
# 对test分词
cleaned_df_test['title_wb'] = cleaned_df_test['title'].progress_apply(tokenizer.cut)
cleaned_df_test['article_wb'] = cleaned_df_test['article'].progress_apply(tokenizer.cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\t-dosu\AppData\Local\Temp\jieba.cache
Loading model cost 0.889 seconds.
Prefix dict has been built succesfully.


D:\software\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


D:\software\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


D:\software\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


D:\software\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## 5.1 根据train数据集构建词典（可选）

In [28]:
counter = collections.Counter()
for title_wb, article_wb, comment_wb in tqdm(zip(sampled_df_train['title_wb'], sampled_df_train['article_wb'], sampled_df_train['comment'])):
    for wb in [title_wb, article_wb, comment_wb]:
        words = wb.split()
        counter.update(words)
counter = counter.most_common()

In [29]:
counter_5w = counter[:50000]

In [30]:
word_dict = dict()
word_dict['<pad>'] = 0
word_dict['<unk>'] = 1
for word, count in counter:
    if count < 20:
        break
    word_dict[word] = len(word_dict)

with open('word_dict.pkl', 'wb') as f:
    pickle.dump(word_dict, f)

In [31]:
print("词汇表的size：",len(word_dict))

词汇表的size： 246037


## 5.2 根据pretrain好的词向量构建词典（可选）

In [17]:
from collections import defaultdict
import numpy as np

# base_path = "D:\\jupyter\\comment_generation\\data\\"
base_path = "D:\\data\\搜狗新闻word2vec\\"
# embeddingFile = "zhwiki_2017_03.sg_50d.word2vec"
embeddingFile = "corpus.vector"
embeddingFile = base_path + embeddingFile

In [18]:
def load_word_dict_ms(filename):
    """
    加载词向量文件

    :param filename: 文件名
    :return: embeddings列表和它对应的索引
    """
    embeddings = []
    embedding_pad = []
    embedding_unk = []
    word2idx = defaultdict(list)
    with open(filename, mode="r", encoding="utf-8") as rf:
        lock = True
        for i,line in enumerate(rf):
            if lock == True:
                word2idx["<pad>"] = 0
                embedding_pad = np.zeros([int(line.split(" ")[1])]).tolist()
                embeddings.append(embedding_pad)
                word2idx["<unk>"] = 1
                embedding_unk = np.random.randn(int(line.split(" ")[1])).tolist()
                embeddings.append(embedding_unk)
                lock = False
                continue
            arr = line.split(" ")
            embedding = [float(val) for val in arr[1: -1]]
            word2idx[arr[0]] = len(word2idx)
            embeddings.append(embedding)
    return embeddings, word2idx

In [19]:
embeddings, word_dict = load_word_dict_ms(embeddingFile)

In [20]:
print(len(word_dict))
word_dict["，"]

556192


2

In [25]:
embeddings[1]

[1.7315808892246911,
 0.37765695288592843,
 -0.3601463003348267,
 -0.34182457699699337,
 -0.47090463717829995,
 -0.15329251036754904,
 0.42138661510802605,
 -0.9462890362592533,
 -2.0562647406916805,
 1.1424566362902449,
 0.5613777172662763,
 -0.1718175895089136,
 0.8701803145599046,
 1.4474447873084195,
 0.348672320789414,
 1.2189740975891634,
 -1.306451158124542,
 0.5164001128889835,
 1.7005991724558,
 0.3704815580858835,
 1.0786154555747505,
 -0.8126139541319054,
 1.0190384169270421,
 -0.07714802279767083,
 -1.6823830047754855,
 1.227803244535351,
 1.6267391677293008,
 -0.2491568971972385,
 1.3672430064294023,
 0.6410397791859909,
 0.40666851546313876,
 0.2228956121959879,
 0.580921025888175,
 -1.2419898499590507,
 2.4145920267461363,
 -0.7953343716153602,
 0.19889062405424876,
 1.0372774363066462,
 -0.020821692591031418,
 -1.56171682418998,
 0.7758649151517705,
 -0.05791491673606784,
 -1.2694734734702868,
 -2.4734306970745625,
 -0.6606563355303575,
 0.8621358031443683,
 -0.96538922

## 6 将 train, dev, test 数据转换成模型需要的id格式

In [32]:
def text_to_ids(word_dict, text_wb):
    words = text_wb.split()
    ids = []
    unk_id = word_dict['<unk>']
    for word in words:
        word_id = word_dict.get(word, unk_id)
        ids.append(word_id)
    return ids


def transform_data(df, save_to, inputs_length=400, outputs_length=100):
    with open(save_to, 'w', encoding='utf-8') as fout:
        for idx, row in tqdm(df.iterrows()):
            title_wb = row['title_wb']
            article_wb = row['article_wb']
            comment_wb = row['comment']
            label = 1 if row['upvote'] >= 10 else 0
            title_ids = text_to_ids(word_dict, title_wb)
            article_ids = text_to_ids(word_dict, article_wb)
            comment_ids = text_to_ids(word_dict, comment_wb)
            inputs = title_ids + article_ids
            inputs = inputs[0: inputs_length]
            inputs = ' '.join(str(it) for it in inputs)
            targets = comment_ids[0: outputs_length]
            targets = ' '.join(str(it) for it in targets)
            fout.write(f'{label}\t{inputs}\t{targets}\n')

In [33]:
# 生成模型需要的训练集
transform_data(sampled_df_train, 'train.csv')

In [34]:
transform_data(cleaned_df_dev, 'dev.csv')

In [35]:
transform_data(cleaned_df_test, 'test.csv')

# 测试

In [89]:
df = pd.DataFrame([1, "", 3, 4, 5], columns=['cols'])
df

,cols
0,1
1,
2,3
3,4
4,5


In [91]:
df[0]

KeyError: 0

In [ ]:
def load_data(filename):
    data = []
    with open(filename, encoding='utf-8') as fin:
        for line in fin:
            item = json.loads(line)
            title = item['title_es'].strip()
            article = item['body'].strip()
            url = item['url']
            uid = url.split('/')[-1]
            for comment in item['comment']:
                comment_content = ''.join(comment[0])
                upvote = int(comment[1])
                data.append({
                    'article_id': uid,
                    'title': title,
                    'article': article,
                    'comment': comment_content,
                    'upvote': upvote
                })
    return pd.DataFrame(data)

filename_train = r'D:\data\comment_generation\article_commenting_learning_comment_generation\newdata.train.json'

In [106]:
su_test_train = load_data(r'D:\data\comment_generation\article_commenting_learning_comment_generation\cleaned_corpus\newdata.train.cleaned.json')

In [116]:
su_grouped = cleaned_df_dev.groupby(['article_id'])

def sample_random_neg(grouped,name,su_grouped_df,count):
    import random
    names = []
    samples_neg = []
    count_name = count
    while(len(names)==0):
        if len(grouped.groups.keys())>count_name:
            names = random.sample(grouped.groups.keys(), count_name)
            if name in names:
                names.remove(name)
        else:
            count_name = len(grouped.groups.keys())- 1
            
    article = su_grouped_df.iat[0,0]
    article_id = su_grouped_df.iat[0,1]
    title = su_grouped_df.iat[0,3]
    upvote = 1
    title_wb = su_grouped_df.iat[0,5]
    article_wb = su_grouped_df.iat[0,6]
    for n in names:
        temp = grouped.get_group(n).sample(n=1, random_state=1)
        temp["article"] = article
        temp["article_id"] = article_id
        temp["title"] = title
        temp["upvote"] = upvote
        temp["title_wb"] = title_wb
        temp["article_wb"] = article_wb
        temp["comment"] = "false"+temp["comment"]
        samples_neg.append(temp)
    return pd.concat(samples_neg)

su_samples = []
for su_name, su_grouped_df in tqdm(su_grouped):
    su_grouped_pos_df = su_grouped_df[su_grouped_df['upvote'] >= 10]
    su_pos_count = len(su_grouped_pos_df)
    if su_pos_count == 0:
        continue
    su_grouped_neg_df = sample_random_neg(su_grouped,su_name,su_grouped_df, su_pos_count)
    su_samples.append(su_grouped_pos_df)
    su_samples.append(su_grouped_neg_df)

su_sampled_df_train = pd.concat(su_samples)

In [106]:
type(su_sampled_df_train)

pandas.core.frame.DataFrame

In [117]:
su_sampled_df_train.groupby(['article_id']).groups.keys()

dict_keys(['20170314A02JSF00', '20170316A07MJN00', '20170328A06AYP00', '20170403A046MQ00', '20170403A065D300', '20170407A01NQ000', '20170407A038G300', '20170407A03E1X00', '20170407A0A1MN00', '20170407A0BA5X00', '20170408A03UW400', '20170408A03X3I00', '20170408A066U600', '20170410A00C2A00', '20170410A0385Y00', '20170410A04GYK00', '20170410A068MJ00', '20170410A0ARPQ00', '20170411A08HCH00', '20170412A02QA300', '20170412A02TI400', '20170412A04SY600', '20170412A05JSG00', '20170412A07TC600', '20170412A08AXD00', '20170412A096GP00', '20170412A0C5JG00', '20170412A0CH5400', '20170412A0CI5W00', '20170412A0CMSU00', '20170412G0C52200', '20170413A02A1R00', '20170413A04KJF00', '20170413A05IDH00', '20170413A06L3M00', '20170413A071AH00', '20170413A079DZ00', '20170413A08DX600', '20170413A08IWY00', '20170413A0ANWB00', '20170413A0C8AA00', '20170414A01BB000', '20170414A01EPH00', '20170414A01P3T00', '20170414A01VJ700', '20170414A024FQ00', '20170414A02EPQ00', '20170414A02ZV900', '20170414A04ZPW00', '20170414

In [123]:
su_sampled_df_train.groupby(['article_id']).get_group("20170622A0082K00")

,article,article_id,comment,title,upvote,title_wb,article_wb
69447,联盟每只球队都拥有自己的核心球员，他们在球队起着至关重要的作用，有的是自家培养的，还要的是花...,20170622A0082K00,还是 那句话 库里 到 哪 ， 心 就 到 哪 。,勇士高层拒绝库里顶薪要求，科尔：这对他不太公平,10,勇士 高层 拒绝 库里 顶薪 要求 ， 科尔 ： 这 对 他 不 太 公平,联盟 每 只 球队 都 拥有 自己 的 核心 球员 ， 他们 在 球队 起着 至关重要 的 ...
69448,联盟每只球队都拥有自己的核心球员，他们在球队起着至关重要的作用，有的是自家培养的，还要的是花...,20170622A0082K00,做 为 球迷 我 希望 库里 离开 勇士 ， 去 胡人 另建 王朝 ， 科尔 讲话 不谈 库...,勇士高层拒绝库里顶薪要求，科尔：这对他不太公平,15,勇士 高层 拒绝 库里 顶薪 要求 ， 科尔 ： 这 对 他 不 太 公平,联盟 每 只 球队 都 拥有 自己 的 核心 球员 ， 他们 在 球队 起着 至关重要 的 ...
69449,联盟每只球队都拥有自己的核心球员，他们在球队起着至关重要的作用，有的是自家培养的，还要的是花...,20170622A0082K00,其实 我 就是 喜欢 库里 而已 ， 库里 可以 去 湖人 和 马刺 ， 整个 赛季 下来 ...,勇士高层拒绝库里顶薪要求，科尔：这对他不太公平,20,勇士 高层 拒绝 库里 顶薪 要求 ， 科尔 ： 这 对 他 不 太 公平,联盟 每 只 球队 都 拥有 自己 的 核心 球员 ， 他们 在 球队 起着 至关重要 的 ...
74985,联盟每只球队都拥有自己的核心球员，他们在球队起着至关重要的作用，有的是自家培养的，还要的是花...,20170622A0082K00,false两个 都 很 好 很 美,勇士高层拒绝库里顶薪要求，科尔：这对他不太公平,1,勇士 高层 拒绝 库里 顶薪 要求 ， 科尔 ： 这 对 他 不 太 公平,联盟 每 只 球队 都 拥有 自己 的 核心 球员 ， 他们 在 球队 起着 至关重要 的 ...
18746,联盟每只球队都拥有自己的核心球员，他们在球队起着至关重要的作用，有的是自家培养的，还要的是花...,20170622A0082K00,false什么 科学家 文学家 企业家 这家 那 家 什么 从商 从政 从医...,勇士高层拒绝库里顶薪要求，科尔：这对他不太公平,1,勇士 高层 拒绝 库里 顶薪 要求 ， 科尔 ： 这 对 他 不 太 公平,联盟 每 只 球队 都 拥有 自己 的 核心 球员 ， 他们 在 球队 起着 至关重要 的 ...
51437,联盟每只球队都拥有自己的核心球员，他们在球队起着至关重要的作用，有的是自家培养的，还要的是花...,20170622A0082K00,false不负责任 的 人 浪费 了 人力物力,勇士高层拒绝库里顶薪要求，科尔：这对他不太公平,1,勇士 高层 拒绝 库里 顶薪 要求 ， 科尔 ： 这 对 他 不 太 公平,联盟 每 只 球队 都 拥有 自己 的 核心 球员 ， 他们 在 球队 起着 至关重要 的 ...


In [92]:
su_grouped = cleaned_df_dev.groupby(['article_id'])

In [93]:
type(su_grouped.groups)

dict

In [45]:
su_grouped.get_group("20161118A07PK500").sample(n=1, random_state=1)[""]

,article,article_id,comment,title,upvote,title_wb,article_wb
97045,持续关注！18日，华盛顿智库亚洲海事透明倡议(Asia Maritime Transpare...,20161118A07PK500,加大 越南 改变现状 的 全球 报道 ， 不要 等到 猴子 闹事 来 阻止 。 让 它 知道...,越南在南海岛礁建跑道，那些批评中国的人哪儿去了？,1,越南 在 南海 岛礁 建 跑道 ， 那些 批评 中国 的 人 哪儿 去 了 ？,持续 关注 ！ 18 日 ， 华盛顿 智库 亚洲 海事 透明 倡议 ( Asia Mar...


In [54]:
import random
print(len(su_grouped.groups.keys()))
random.sample(su_grouped.groups.keys(), 5000000000)

4434


ValueError: Sample larger than population or is negative

In [67]:
a = [1,2,3,4]
xx = 9

if xx in a:
    
    a.remove(xx)
a
# a.remove(1)
# print(a)

[1, 2, 3, 4]

In [4]:
import numpy as np
a = np.random.randn(300).tolist()
len(a)

300